# Model Training - Time Series Cross-Validation
A number combination of different model architectures, features used, and training approaches were tested in the [04a-modeling-classic-cv.ipynb](./04a-modeling-classic-cv.ipynb) notebook. The results of that analysis was used to inform further model architectural testing. Since the baseline models' performance isn't expected to change, they won't be tested in this notebook. Similarly, since the 7 lasso features did not seem to improve model performance, they will not be tested in this notebook and instead all 67 features will be used.

## Model Architectures
1. [Linear Regression Model](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
1. [Random Forest Model](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)
1. [XGBoost Model](https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.XGBRegressor)

## Features
1. All 67 features
    - 31 categorical one-hot features for teams (30 MLB teams and '---' for multi-team)
    - 36 numeric features

## Training Approach
- Time-series cross-validation approach (use `2021` to predict `2022` and `2021-2022` to predict `2023`)
- See [02-data-partitioning.ipynb](./02-data-partitioning.ipynb) for more details

**THIS NOTEBOOK USES THE TIME SERIES CROSS-VALIDATION APPROACH**
The time series cross-validation approach will be used to test the various combinations for model architectures listed above.

In all cases, the `MSE` (mean squared error) is used to evaluate model performance.

**NOTE**: All pipelines and functions defined in this notebook are defined in source code at `bullpen.model_utils`. They are show here for convenience, but stored in source code for future use and ability to unit test.

In [1]:
from itertools import product

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import scipy.stats
import xgboost as xgb
from sklearn import set_config
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

set_config(display="text")

from bullpen import data_utils, model_utils

In [2]:
train_df = pd.read_csv(data_utils.DATA_DIR.joinpath("train.csv"))
train_df

,MLBAMID,PlayerId,Name,Team,Age,Season,TBF,K%,Rk,IP,...,02s,02h,L/SO,S/SO,L/SO%,3pK,4pW,PAu,Pitu,Stru
0,621345,18655,A.J. Minter,ATL,27,2021,221,0.257919,696,52.1,...,44,7,11,46,0.192982,11,4,0,0,0
1,621345,18655,A.J. Minter,ATL,28,2022,271,0.346863,649,70.0,...,50,2,23,71,0.244681,12,0,0,0,0
2,621345,18655,A.J. Minter,ATL,29,2023,260,0.315385,647,64.2,...,40,4,13,69,0.158537,8,1,0,0,0
3,640462,19343,A.J. Puk,OAK,27,2022,281,0.270463,773,66.1,...,48,6,22,54,0.289474,15,4,0,0,0
4,640462,19343,A.J. Puk,MIA,28,2023,242,0.322314,755,56.2,...,42,6,22,56,0.282051,16,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,425844,1943,Zack Greinke,HOU,37,2021,697,0.172166,417,171.0,...,51,4,34,85,0.285714,13,6,0,0,0
984,425844,1943,Zack Greinke,KCR,38,2022,585,0.124786,396,137.0,...,39,3,22,51,0.301370,7,2,0,0,0
985,425844,1943,Zack Greinke,KCR,39,2023,593,0.163575,353,142.1,...,53,6,25,70,0.263158,11,2,0,0,0
986,668868,25918,Zack Thompson,STL,24,2022,136,0.198529,967,34.2,...,40,3,10,17,0.370370,3,2,0,0,0


In [3]:
year_list = train_df.Season.unique().tolist()
year_list

[2021, 2022, 2023]

In [4]:
def make_timeseries_splits(year_list, train_df):
    splits = {"train": [], "val": []}
    for idx, year in enumerate(year_list[:-1]):
        train_years = year_list[: idx + 1]
        val_year = year_list[idx + 1]

        print(f"TRAIN: {train_years} VAL: {[val_year]}")

        splits["train"].append(train_df[train_df["Season"].isin(train_years)])
        splits["val"].append(train_df[train_df["Season"] == val_year])
    return splits

In [5]:
splits = make_timeseries_splits(year_list, train_df)

TRAIN: [2021] VAL: [2022]
TRAIN: [2021, 2022] VAL: [2023]


In [6]:
# sniff test (split 1)
splits["train"][0].Season.unique(), splits["val"][0].Season.unique()

(array([2021]), array([2022]))

In [7]:
# sniff test (split 2)
splits["train"][1].Season.unique(), splits["val"][1].Season.unique()

(array([2021, 2022]), array([2023]))

In [8]:
def pred_X_y(split, target="K%", drop_cols=None):
    drop_cols = ["Name", target] if drop_cols is None else drop_cols

    X_df = split[[c for c in split.columns if c not in drop_cols]]
    y_df = split[target]
    return X_df, y_df

In [9]:
X_df, y_df = pred_X_y(splits["train"][0])
X_val_df, y_val_df = pred_X_y(splits["val"][0])

In [10]:
# sniff test
X_df.Season.unique(), X_val_df.Season.unique()

(array([2021]), array([2022]))

In [11]:
X_df, y_df = pred_X_y(splits["train"][1])
X_val_df, y_val_df = pred_X_y(splits["val"][1])

In [12]:
# sniff test
X_df.Season.unique(), X_val_df.Season.unique()

(array([2021, 2022]), array([2023]))

In [13]:
processor = model_utils.make_processing_pipeline(
    categorical_features=["Team"],
    numeric_features=[f for f in X_df.columns if f not in ("Team")],
)

processor

ColumnTransformer(transformers=[('categorical',
                                 Pipeline(steps=[('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Team']),
                                ('numeric',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['MLBAMID', 'PlayerId', 'Age', 'Season', 'TBF',
                                  'Rk', 'IP', 'PA', 'Pit', 'Pit/PA', 'Str',
                                  'Str%', 'L/Str', 'S/Str', 'F/Str', 'I/Str',
                                  'AS/Str', 'I/Bll', 'AS/Pit', 'Con', '1st%',
                                  '30%', '30c', '30s', '02%', '02c', '02s',
                                  '02h', 'L/SO', 'S/SO', ...])])

## Model training

### Baseline Estimators
Sine the results of the baseline models will not change (predictions based solely on the target variable) and since the `ArticleModel ` used raw, unscaled data, they were left out of model training in this notebook.
- The results aren't expected to change much, if at all
- Models trained with the manually time-series cross-validation approach should still beat these baseline models:

|                       |        R2 | 	     mse|
|-----------------------|-----------|-----------|
|ArticleModel()	        |   0.869021|	0.000417|
|Baseline(method='mean')|	0.834678|	0.000527|
|Baseline(method='last')|	0.673340|	0.001041|


### Non-Baseline Estimators
Now that baselines have been established, more advanced models are create: 
1. [Linear Regression Model](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
1. [Random Forest Model](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)
1. [XGBoost Model](https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.XGBRegressor)

For each of the above model architectures, the different feature sets will be tested (All 67 features and 7 lasso features):
1. All 67 features
    - 31 categorical one-hot features for teams (30 MLB teams and '---' for multi-team)
    - 36 numeric features
1. 7 features selected from Lasso model (see [03-feature-engineering.ipynb](./03-feature-engineering.ipynb))
   - `'numeric__Pit/PA'`
   - `'numeric__Str%'`
   - `'numeric__F/Str'`
   - `'numeric__I/Str'`
   - `'numeric__Con'`
   - `'numeric__30%'`
   - `'numeric__L/SO'`

**NOTE**: The `Baseline` models never used the features to make predictions (they only took the mean or last data point from the target variable (`'K%'`). The `ArticleModel` used specific columns with the highest predictive power in the article. Thus, testing performance on two different feature sets didn't make sense. For the remainder of the models, performance will be observed for both the full feature set and the limited lasso feature set.

### Time Series Cross Validation (TODO)

In [14]:
def cross_validate_model(
    model, param_grid, splits, processor, metric_key="mean_mse", K=2
):
    """
    Manual cross-validation based on custom timeseries data
    """
    results = []
    param_names = list(param_grid.keys())
    param_combinations = list(product(*param_grid.values()))

    for params in param_combinations:
        param_dict = dict(zip(param_names, params))
        print(f"Testing parameters: {param_dict}")

        split_scores = []
        for split_idx in range(K):
            # Get training and validation data
            X_df, y_df = pred_X_y(splits["train"][split_idx])
            X_val_df, y_val_df = pred_X_y(splits["val"][split_idx])
            print(f"TRAIN: {X_df.Season.unique()} VAL: {X_val_df.Season.unique()}")

            # Initialize and train the model
            preds, metrics = model_utils.train_model(
                processor, model(**param_dict), X_df, y_df, results={}, name="model"
            )

            # Collect the desired metric (e.g., MSE) which is the second, or last appended
            split_scores.append(metrics["model"][-1])

        # Compute mean metric across splits
        mean_metric = np.mean(split_scores)
        results.append({**param_dict, metric_key: mean_metric})

        print(f"Mean {metric_key}: {mean_metric:.4f}")
        print()

    # Find the best hyperparameters based on the lowest metric
    best_result = min(results, key=lambda x: x[metric_key])
    return results, best_result

In [15]:
lr_param_grid = {"fit_intercept": [True, False]}

lr_results, lr_best_result = cross_validate_model(
    model=LinearRegression,
    param_grid=lr_param_grid,
    splits=splits,
    processor=processor,
)

print("Best Hyperparameters for Linear Regression:")
print(lr_best_result)

Testing parameters: {'fit_intercept': True}
TRAIN: [2021] VAL: [2022]
model params=None score=0.954 mse=0.00014
TRAIN: [2021 2022] VAL: [2023]
model params=None score=0.951 mse=0.00016
Mean mean_mse: 0.0002

Testing parameters: {'fit_intercept': False}
TRAIN: [2021] VAL: [2022]
model params=None score=0.954 mse=0.00014
TRAIN: [2021 2022] VAL: [2023]
model params=None score=0.951 mse=0.00016
Mean mean_mse: 0.0002

Best Hyperparameters for Linear Regression:
{'fit_intercept': True, 'mean_mse': 0.00015145634315338836}


In [16]:
rf_param_grid = {"n_estimators": [25, 50, 100, 150], "max_depth": [5, 10, 15]}

rf_results, rf_best_result = cross_validate_model(
    model=RandomForestRegressor,
    param_grid=rf_param_grid,
    splits=splits,
    processor=processor,
)

print("Best Hyperparameters for RandomForestRegressor:")
print(rf_best_result)

Testing parameters: {'n_estimators': 25, 'max_depth': 5}
TRAIN: [2021] VAL: [2022]
model params=None score=0.953 mse=0.00015
TRAIN: [2021 2022] VAL: [2023]
model params=None score=0.943 mse=0.00019
Mean mean_mse: 0.0002

Testing parameters: {'n_estimators': 25, 'max_depth': 10}
TRAIN: [2021] VAL: [2022]
model params=None score=0.980 mse=0.00006
TRAIN: [2021 2022] VAL: [2023]
model params=None score=0.981 mse=0.00006
Mean mean_mse: 0.0001

Testing parameters: {'n_estimators': 25, 'max_depth': 15}
TRAIN: [2021] VAL: [2022]
model params=None score=0.982 mse=0.00006
TRAIN: [2021 2022] VAL: [2023]
model params=None score=0.984 mse=0.00005
Mean mean_mse: 0.0001

Testing parameters: {'n_estimators': 50, 'max_depth': 5}
TRAIN: [2021] VAL: [2022]
model params=None score=0.957 mse=0.00014
TRAIN: [2021 2022] VAL: [2023]
model params=None score=0.943 mse=0.00019
Mean mean_mse: 0.0002

Testing parameters: {'n_estimators': 50, 'max_depth': 10}
TRAIN: [2021] VAL: [2022]
model params=None score=0.983 

In [17]:
xgb_param_grid = {"n_estimators": [25, 50, 100, 150], "max_depth": [5, 10, 15]}

xgb_results, xgb_best_result = cross_validate_model(
    model=xgb.XGBRegressor,
    param_grid=xgb_param_grid,
    splits=splits,
    processor=processor,
)

print("Best Hyperparameters for XGBRegressor:")
print(xgb_best_result)

Testing parameters: {'n_estimators': 25, 'max_depth': 5}
TRAIN: [2021] VAL: [2022]
model params=None score=0.998 mse=0.00001
TRAIN: [2021 2022] VAL: [2023]
model params=None score=0.992 mse=0.00003
Mean mean_mse: 0.0000

Testing parameters: {'n_estimators': 25, 'max_depth': 10}
TRAIN: [2021] VAL: [2022]
model params=None score=1.000 mse=0.00000
TRAIN: [2021 2022] VAL: [2023]
model params=None score=1.000 mse=0.00000
Mean mean_mse: 0.0000

Testing parameters: {'n_estimators': 25, 'max_depth': 15}
TRAIN: [2021] VAL: [2022]
model params=None score=1.000 mse=0.00000
TRAIN: [2021 2022] VAL: [2023]
model params=None score=1.000 mse=0.00000
Mean mean_mse: 0.0000

Testing parameters: {'n_estimators': 50, 'max_depth': 5}
TRAIN: [2021] VAL: [2022]
model params=None score=1.000 mse=0.00000
TRAIN: [2021 2022] VAL: [2023]
model params=None score=0.999 mse=0.00000
Mean mean_mse: 0.0000

Testing parameters: {'n_estimators': 50, 'max_depth': 10}
TRAIN: [2021] VAL: [2022]
model params=None score=1.000 

## Conclusion
- Each model continued to show good results with the Time Series cross-validation (it wasn't expected to be different, but good to know).
- While the tree based models (`RandomForestRegressor`, `XGBRegressor`) out performed the linear model, the interpretability and ease-of-use of the linear model is an attractive aspect. For this reason, one of the tree-based models (`XGBRegressor`) and the linear model (`LinearRegression`) will be saved and used to make final predictions.

In [18]:
models = ("LinearRegression", "RandomForestRegressor", "XGBRegressor")
mses = [d["mean_mse"] for d in [lr_best_result, rf_best_result, xgb_best_result]]
results_df = (
    pd.DataFrame(zip(models, mses), columns=["model", "mse"])
    .set_index("model")
    .sort_values("mse")
    .round(6)
)
results_df

,mse
model,
XGBRegressor,0.000000
RandomForestRegressor,0.000048
LinearRegression,0.000151
